In [1]:
import pandas as pd
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', -1)

**Read in file**

In [2]:
column_names=['Name', 'ScreenName', 'UserID', 'FollowersCount', 'FriendsCount', 'Location', 'Description', 'CreatedAt', 'StatusID', 'Language', 'Place', 'RetweetCount', 'FavoriteCount', 'Text']
tweets = pd.read_csv('clinton_trump_tweets.txt', sep="\t",  encoding="ISO-8859-1", header=None, names=column_names)
#print initial data
tweets.head(20)

,Name,ScreenName,UserID,FollowersCount,FriendsCount,Location,Description,CreatedAt,StatusID,Language,Place,RetweetCount,FavoriteCount,Text
0,Cebel,Cebel6,1519696717,132,263,"Little Rock, Arkansas",Arkansas Razorback Fan Just trying to be #Uncommon one 1-0 day at a time.,Sat Oct 29 08:10:06 EEST 2016,792232017094119425,en,NaN,0,1,@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
1,Cookie,Cookiemuffen,109945090,2154,2034,The American South,Got married after college. I don't regret starting a family instead of grad school. Proud Deplorable,Wed Oct 26 18:44:08 EEST 2016,791304413923213312,en,NaN,1937,0,RT @wikileaks: New poll puts Pirate Party on course to win Iceland's national elections on Saturday. https://t.co/edTqjeJaQ6
2,nolaguy,nolaguy_phd,1450086582,797,1188,NaN,"An LSU Ph.D student living in New Orleans, trying to find a second act.",Sat Oct 29 21:53:29 EEST 2016,792439227090767872,en,NaN,0,0,@gaystoner821 I think New Orleans spoiled me with food. I need to try and branch out in BR.
3,Mark Hager,marksnark,167177185,204,448,Pittsburgh,"Hip, trendy, smart, funny, fit, lobbyist. U? Boilerplate: these thoughts are my own, not anyone else's. Hmmmkay?",Wed Oct 26 00:33:20 EEST 2016,791029904733331457,en,NaN,891,0,RT @LOLGOP: ACA needs fixes but know da facts: *70% can get covered in marketplaces for under $75/month *Hikes affect 3% *GOP will uninsu
4,Capitalist Creations,aaronjhoddinott,1191022351,775,154,Canada,"Entrepreneur, startup investor, political junkie, free market supporter, beer connoisseur, dad and dog lover. Also a golf enthusiast despite my lack of skill.",Fri Oct 28 05:05:10 EEST 2016,791823089700962304,en,NaN,7,0,RT @FastCompany: Alphabet shares soar on better-than-expected earnings as mobile video strategy pays off https://t.co/bokbXngMJt https://t.
5,Josh,BigPapiCarnivor,154393802,646,2985,NaN,"Embrace the feel, induce the euphoria, and just stunt",Sat Oct 29 02:34:44 EEST 2016,792147620986839040,en,NaN,51,0,RT @Pattric: That one follower who tweet you everyday and even tho you don't respond they still be @'ing you https://t.co/QBeQyc2PO6
6,David Walling,davidjwalling,106568768,975,2781,"Dallas, TX",Bloodletting secure algorithms close to the bone. #HealthIT security matters. Opinions my own. https://t.co/Dcpe6FteOq,Sat Oct 29 00:16:48 EEST 2016,792112907488079872,en,NaN,0,0,#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv
7,robert2266,robert2266,17101060,845,938,The Universe,The Dark Lord,Fri Oct 28 14:41:06 EEST 2016,791968028191711237,en,NaN,0,0,"Hacked e-mails show Clinton campaigns fears about Sanders | https://t.co/WMyCHuCDIc The Philippine Star (PhilippineStar) October 28, 2"
8,Brianna N. Love,Brianna__Loves,17352518,803,1280,#thesea,green blogger|Swells'19 architecture and geoscience major| IG: @Brianna__Loves| Business inquiry thesealt@gmail.com| https://t.co/SeF8NMVNjS| #thesea,Fri Oct 28 07:32:18 EEST 2016,791860117671124992,en,NaN,24,0,RT @africaarguments: This year's seen some remarkable new films by African directors. Here are a few of them. https://t.co/9JEY4ubGVE http
9,Polly Ticks,PTicks,2352007196,1422,863,everywhere and nowhere,"a girl interested in the political world, like to share my opinions, thoughts, don't lean left or right, bit libertarian, value all practical thoughts",Fri Oct 28 19:01:13 EEST 2016,792033486970454020,en,NaN,3,0,"RT @HalifaxEditor: Some 40 years later, @HfxRegPolice continue to investigate the murder of Florence Ann Keeble. https://t.co/axdvA1HUcu ht"


**Filter out retweets*
Tweets that start with 'RT'

In [3]:
#1.1 filter retweets
tweets = tweets[~tweets.Text.str.startswith("RT")]
tweets.Text.str.findall('.*?(http\:\/\/www\.[a-zA-Z0-9\.\/\-]+)')

0          []
2          []
6          []
7          []
10         []
12         []
13         []
18         []
20         []
21         []
22         []
24         []
25         []
27         []
28         []
30         []
32         []
33         []
34         []
35         []
36         []
38         []
40         []
43         []
49         []
51         []
52         []
53         []
54         []
55         []
           ..
5250920    []
5250922    []
5250923    []
5250925    []
5250928    []
5250930    []
5250931    []
5250934    []
5250936    []
5250937    []
5250938    []
5250939    []
5250942    []
5250944    []
5250945    []
5250947    []
5250948    []
5250953    []
5250954    []
5250956    []
5250959    []
5250963    []
5250965    []
5250966    []
5250967    []
5250968    []
5250973    []
5250975    []
5250977    []
5250978    []
Name: Text, Length: 2416530, dtype: object

In [4]:
def keepHandlesAndMention(text):
    handlesAndMentions = []
    words =  text.split()
    for word in words:
        if((word.startswith('@') or word.startswith('#')) and len(word) > 1):
            handlesAndMentions.append(word)
    return ' '.join(handlesAndMentions)
tweets['TextMentions'] = tweets.Text.apply(keepHandlesAndMention)

**Filter out tweets that have mention/hashtag < 20

In [ ]:
allHandles=[]
for text in tweets.TextMentions:
    allHandles.extend(text.split())
top_hash = pd.Series(allHandles).value_counts()
top20 = top_hash[top_hash>=20]
top20List = top20.index.tolist()


In [ ]:
# tweets[tweets.TextMentions.str.map(lambda item: item.split(',').foreach(x: x in ))]
def isInList(list1,list2):
    for item1 in list1:
        if(item1 in list2 ):
            return True
    return False
tweets = tweets[tweets.TextMentions.apply(lambda x : any(elem in top20List for elem in x))]


In [ ]:
#1.1 remove tweets that don't have any @ or #
#tweets["CleanText"] = tweets.Text.str.extract('((@|#)\w+)')
#use this to get all @ # from tweet, extract just gets first occurance
#.groupby(level=0)[0].apply(' '.join)
# tweets = tweets[tweets.CleanText.notnull()]
# tweets.head()

**Filter out users that have less than 20 tweets**

In [ ]:
#1.2 keep tweets where UID appears 20+ times
tweets = tweets.groupby("UserID").filter(lambda x: len(x) >= 20)
tweets

**Plot top 30 locations

In [ ]:
#2.1
tweetsTop30Locations = tweets[tweets.Location.isin(tweets.Location.value_counts().nlargest(30).index.tolist())]

In [ ]:
tweetsTop30Locations.groupby("Location").Location.count().plot(kind='bar',figsize=(20,10))

**Make word cloud of the top 3 locations

In [ ]:
tweetsTop3Locations = tweets[tweets.Location.isin(tweets.Location.value_counts().nlargest(3).index.tolist())]
text = " ".join(tweet for tweet in tweetsTop3Locations.Text)


In [ ]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["go","ye","thing","well","big","us","great","https", "still", "need", "co", "one","will","Thank","know","going","lol","good", "take","even","really","now"])


# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords).generate(text)

# Display the generated image:
plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# tweetsTop50Users = tweets[tweets.UserID.isin(tweets.UserID.value_counts().nlargest(50).index.tolist())]
tweetsTop50Users = tweets[tweets.UserID.isin(tweets.UserID.value_counts().nlargest(50).index.tolist())]

In [ ]:
tweetsTop50Users.groupby("ScreenName").UserID.count().plot(kind='bar',figsize=(20,10))

In [ ]:
tweetsTop3Users = tweets[tweets.UserID.isin(tweets.UserID.value_counts().nlargest(3).index.tolist())]
top3UsersText = " ".join(tweet for tweet in tweetsTop3Users.Text)


In [ ]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["YOUNGMA", "https", "co", "DOWNLOAD", "CJRr8Y9xHM"])


# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords).generate(top3UsersText)

# Display the generated image:
plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
tweets.UserID.value_counts().nlargest(3)